# New York Versus Los Angeles

## TOC:
* [Introduction](#Introduction)
* [Data](#Data)
* [Methodology](#Methodology)
* [Results & Observations](#Results&Observations)

## Introduction <a class="anchor" id="Introduction"></a>

Many people have aspirations to live in some of the biggest cities in world. Some countries have multiple amazing modern cities with amazing amenities and opportunities, so it can be hard to decide which of cities to choose. We’re going to look at two of major metropolises in the United States: New York City and Los Angeles. We are going to compare the different aspects of these two cities to find out which one is right for you. We are going to compare the different neighbourhoods in both cities in terms of the available amenities and places you can go to in the different cities. We will find out if these two cities are similar or actually quite different.

## Data <a class="anchor" id="Data"></a>

The data we will be using for this report will be a dataset of the different neighbourhoods in the two cities as well as the different places in the two cities which we will acquire from the Wikipedia page:  and the foursquare API for collecting location venue data. We will also use Geopy to get coordiantes for these different venues. Using the data, we will group the neighbourhood into similar clusters and we will see if the neighbourhoods from the different cities fall into similar clusters or different clusters. This should give a sense of the similarities between the two cities.

## Methodology <a class="anchor" id="Methodology"></a>

Let's download all the dependencies that we will need

In [77]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip install geopandas
!pip install geopy

! pip install requests

! pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


For our New York data, we already have it on an IBM server so we just download it and create a dataset of the neighbourhoods with their latitudes and longitudes.

In [78]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


Now let's load the data

In [79]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

All the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [80]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [81]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a _pandas_ dataframe

The next task is essentially transforming this data of nested Python dictionaries into a _pandas_ dataframe. So let's start by creating an empty dataframe.

In [82]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Then let's loop through the data and fill the dataframe one row at a time.

In [83]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [84]:
new_york_data = neighborhoods
new_york_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [85]:
new_york_data.shape

(306, 4)

We then create a map of New York City with the neighborhoods superimposed on top

In [86]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [87]:
map_new_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(new_york_data['Latitude'], new_york_data['Longitude'], new_york_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_new_york)  
    
map_new_york

For our Los Angeles data, We scrape this data from Wikipedia page https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_in_Los_Angeles using BeautifulSoup

In [88]:
link = requests.get("https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_in_Los_Angeles")
soup = BeautifulSoup(link.text, "html.parser")

sections=soup.find_all("div", class_="div-col")
places = BeautifulSoup(str(sections)).find_all('li')

neighborhoods_list = []

for div in places:
    if div.find('a').contents[0] == '[40]':
        neighborhoods_list.append('Pico Robertson')
    else:
        neighborhoods_list.append(div.find('a').contents[0])
        

We have a mistake, we need to remove the value'[41]' from the list

In [89]:
neighborhoods_list.remove('[41]')

In [90]:
len(neighborhoods_list)

200

We have 200 neighbourhoods as expected

In [91]:
locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('{}, Los Angeles, California'.format('Angelus Vista'))

In [92]:
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

Latitude = -23.403598000000002, Longitude = -51.96581770186097


We now use geopy to get the coordinates of the neighbourhoods and we put our data in a data frame

In [93]:
Los_Angeles_data = pd.DataFrame(columns=['Neighborhood','Latitude','Longitude'])

In [94]:
for hood in neighborhoods_list:
    locator = Nominatim(user_agent='myGeocoder')
    location = locator.geocode('{}, Los Angeles, California'.format(hood))
    if location == None:
        print(hood)
        continue
    latitude = location.latitude
    longitude = location.longitude
    row =  {'Neighborhood': hood, 'Latitude': latitude, 'Longitude' : longitude}
    Los_Angeles_data = Los_Angeles_data.append(row, ignore_index=True)
    


Baldwin Vista
Bel Air, Bel-Air or Bel Air Estates
Brentwood Glen
Canterbury Knolls
Faircrest Heights
Filipinotown, Historic
Hollywood Dell
Holmby Hills
L.A. Downtown Industrial District
Kinney Heights
Little Ethiopia
NoHo Arts District
Outpost Estates
Picfair Village
Yucca Corridor


We failed to get some coordinates for some of the neighborhoods so we ignore these and we have a look at the data frame we just created.

In [95]:
Los_Angeles_data.head()

,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070289,-118.254796
1,Angeles Mesa,33.991402,-118.319520
2,Angelus Vista,-23.403598,-51.965818
3,Arleta,34.241327,-118.432205
4,Arlington Heights,34.043494,-118.321374


We see that we have some odd coordinates for 'Angelus vista' neighborhood so we shall also be removing this from our data frame as well

In [96]:
Los_Angeles_data = Los_Angeles_data.drop(labels=2, axis=0)
Los_Angeles_data = Los_Angeles_data.reset_index(drop=True)
Los_Angeles_data.head()

,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070289,-118.254796
1,Angeles Mesa,33.991402,-118.319520
2,Arleta,34.241327,-118.432205
3,Arlington Heights,34.043494,-118.321374
4,Arts District,34.041239,-118.234450


Our dataframe looks good now.

In [97]:
Los_Angeles_data.shape

(184, 3)

We then create a map of Los Angeles with the neighborhoods superimposed on top

In [98]:
latitude = 34.0522
longitude = -118.2437

In [99]:
map_Los_Angeles = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Los_Angeles_data['Latitude'], Los_Angeles_data['Longitude'], Los_Angeles_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Los_Angeles)  
    
map_Los_Angeles

We are going to put all our neighbourhoods into one big dataframe (neighborhods) and find all the venues in these neighbourhoods using the Foursquare API. We will then run K-means clustering on these venues to see if they fall into similar clusters.

We want to add a city column to both data frames so we still keep track of the city each neighborhood belongs to.


In [100]:
city = []
for i in range(306):
    city.append('New York')
new_york_data['City'] = city

In [101]:
city = []
for i in range(184):
    city.append('Los Angeles')
Los_Angeles_data['City'] = city

In [102]:
new_york_data.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Bronx,Wakefield,40.894705,-73.847201,New York
1,Bronx,Co-op City,40.874294,-73.829939,New York
2,Bronx,Eastchester,40.887556,-73.827806,New York
3,Bronx,Fieldston,40.895437,-73.905643,New York
4,Bronx,Riverdale,40.890834,-73.912585,New York


In [103]:
Los_Angeles_data.head()

,Neighborhood,Latitude,Longitude,City
0,Angelino Heights,34.070289,-118.254796,Los Angeles
1,Angeles Mesa,33.991402,-118.319520,Los Angeles
2,Arleta,34.241327,-118.432205,Los Angeles
3,Arlington Heights,34.043494,-118.321374,Los Angeles
4,Arts District,34.041239,-118.234450,Los Angeles


We also don't need the borough information for the New York data so we will drop that column

In [104]:
new_york_data.drop(columns = ['Borough'], inplace=True)

In [105]:
new_york_data.head()

,Neighborhood,Latitude,Longitude,City
0,Wakefield,40.894705,-73.847201,New York
1,Co-op City,40.874294,-73.829939,New York
2,Eastchester,40.887556,-73.827806,New York
3,Fieldston,40.895437,-73.905643,New York
4,Riverdale,40.890834,-73.912585,New York


In [106]:
frames = [new_york_data, Los_Angeles_data]

neighborhoods = pd.concat(frames)

In [107]:
neighborhoods.shape

(490, 4)

So we have 490 neighborhoods in total

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version


In [108]:
CLIENT_ID = 'NNSDMQOBQPBSIHOCMMLV1UWVLF53TQ0FGYWGET5QD2NWEKPS' # your Foursquare ID
CLIENT_SECRET = 'HHLEYS132WE0AKKZVASZI4YRWZTE2KIKLTSYMQ4IVVSNRRHX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NNSDMQOBQPBSIHOCMMLV1UWVLF53TQ0FGYWGET5QD2NWEKPS
CLIENT_SECRET:HHLEYS132WE0AKKZVASZI4YRWZTE2KIKLTSYMQ4IVVSNRRHX


#### Let's create a function to get the venues in the neighborhoods


In [109]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _neighborhood_venues_.


In [110]:
neighborhood_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

KeyError: 'groups'

#### Let's check the size of the resulting dataframe


In [37]:
print(neighborhood_venues.shape)
neighborhood_venues.head()

(14891, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


Let's see how many venues were returned by each neighbourhood

In [38]:
neighborhood_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,32,32,32,32,32,32
Angeles Mesa,7,7,7,7,7,7
Angelino Heights,26,26,26,26,26,26
Annadale,13,13,13,13,13,13
Arden Heights,5,5,5,5,5,5
Arleta,3,3,3,3,3,3
Arlington,5,5,5,5,5,5
Arlington Heights,11,11,11,11,11,11
Arrochar,21,21,21,21,21,21


#### Let's find out how many unique categories can be curated from all the returned venues


In [39]:
print('There are {} uniques categories.'.format(len(neighborhood_venues['Venue Category'].unique())))

There are 492 uniques categories.


We now analyze each neighborhood 

In [40]:
# one hot encoding
neighborhood_onehot = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhood_onehot['Neighborhood'] = neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhood_onehot.columns[-1]] + list(neighborhood_onehot.columns[:-1])
neighborhood_onehot = neighborhood_onehot[fixed_columns]

neighborhood_onehot.head()

,Yoshoku Restaurant,ATM,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Terminal,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat Launch,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Caucasian Restaurant,Cemetery,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Quad,College Rec Center,College Residence Hall,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Mark

Let's see the new dataframe size

In [41]:
neighborhood_onehot.shape

(14891, 492)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [42]:
neighborhood_grouped = neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
neighborhood_grouped.head(10)

,Neighborhood,Yoshoku Restaurant,ATM,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Terminal,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat Launch,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Caucasian Restaurant,Cemetery,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Quad,College Rec Center,College Residence Hall,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Di

In [43]:
neighborhood_grouped.shape

(474, 492)

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhood_grouped['Neighborhood']

for ind in np.arange(neighborhood_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Supermarket,Deli / Bodega,Pizza Place,Chinese Restaurant,Pharmacy,Discount Store,Intersection,Check Cashing Service,Grocery Store,Martial Arts School
1,Angeles Mesa,Playground,Wine Bar,Fast Food Restaurant,Deli / Bodega,Seafood Restaurant,Restaurant,Empanada Restaurant,Electronics Store,English Restaurant,Fabric Shop
2,Angelino Heights,Mexican Restaurant,Taco Place,Bakery,Historic Site,Clothing Store,Market,Cocktail Bar,Coffee Shop,Performing Arts Venue,BBQ Joint
3,Annadale,Pizza Place,Juice Bar,Sushi Restaurant,Cosmetics Shop,Food,Pub,Diner,Dance Studio,American Restaurant,Train Station
4,Arden Heights,Playground,Pharmacy,Business Service,Coffee Shop,Pizza Place,Entertainment Service,English Restaurant,Escape Room,Falafel Restaurant,Empanada Restaurant


#### Cluster Neighborhoods

Run K-means to cluster the neighborhoods into 10 clusters

In [64]:
# set number of clusters
kclusters = 10

neighborhood_grouped_clustering = neighborhood_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([8, 8, 0, 8, 8, 0, 8, 9, 8, 0], dtype=int32)

In [65]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhood_merged = neighborhoods

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
neighborhood_merged = neighborhood_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

neighborhood_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,40.894705,-73.847201,New York,8.0,Pharmacy,Caribbean Restaurant,Donut Shop,Sandwich Place,Dessert Shop,Deli / Bodega,Ice Cream Shop,Laundromat,Escape Room,Factory
1,Co-op City,40.874294,-73.829939,New York,8.0,Bus Station,Fast Food Restaurant,Pizza Place,Bagel Shop,Basketball Court,Park,Donut Shop,Liquor Store,Pharmacy,Discount Store
2,Eastchester,40.887556,-73.827806,New York,8.0,Bus Station,Caribbean Restaurant,Deli / Bodega,Diner,Cosmetics Shop,Bowling Alley,Bus Stop,Fast Food Restaurant,Chinese Restaurant,Seafood Restaurant
3,Fieldston,40.895437,-73.905643,New York,8.0,Plaza,Medical Supply Store,Bus Station,River,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Farm,Ethiopian Restaurant
4,Riverdale,40.890834,-73.912585,New York,8.0,Bus Station,Park,Home Service,Bank,Medical Supply Store,Baseball Field,Playground,Plaza,Gym,Food Truck


Some neighborhoods didn't have any venues returned by the Foursquare API so they got a cluster label of NaN so we remove these neighborhoods

In [66]:
neighborhood_merged.dropna(subset = ["Cluster Labels"], inplace=True)

Finally, let's visualize the resulting clusters


We first look at the Los Angeles map

In [111]:
latitude = 34.0522
longitude = -118.2437

In [112]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhood_merged['Latitude'], neighborhood_merged['Longitude'], neighborhood_merged['Neighborhood'], neighborhood_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [113]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [114]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhood_merged['Latitude'], neighborhood_merged['Longitude'], neighborhood_merged['Neighborhood'], neighborhood_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Results & Observations <a class="anchor" id="Results&Observations"></a>

If we look at both of our maps, we see that majority of the neighborhoods in both cities fall into the same clusters. There really isn't a difference in the combination of clusters in both cities. This shows that majority of the neighborhoods are quite similar with similar venues and amenities. So if you want to to choose between living in New York and Los Angeles, based on the venues, their isn't much difference, if you prefer the East Coast and some winter, pick New York, if you prefer the West Coast and warmer all year round, then Los Angeles is for you. 